In [1]:
import itertools
import pandas as pd
import statistics

pd.options.mode.chained_assignment = None  # default='warn'
    
def calculateKendallTauDistance(listA, listB):
    distance = 0
    combinations = []
    for a in listA:
        for b in listB:
            if a != b:
                combinations.append(sorted([a, b]))
    pairs = list(set(map(tuple, combinations)))
    for x, y in pairs:
        a = listA.index(x) - listA.index(y)
        b = listB.index(x) - listB.index(y)
        if a * b < 0:
            distance += 1
    return distance

def partialScoring(dropOutOrder, playerPredictionsArray):
    placeHolders1 = list(map(chr, range(97, 97 + len(playerPredictionsArray) - 1)))
    placeHolders2 = list(map(chr, range(97, 97 + len(playerPredictionsArray) - 1)))
    
    predictionsWithPlaceHolders = []
    dropsWithPlaceHolders = []
    
    for candidate in playerPredictionsArray:
        if candidate not in dropOutOrder: # candidate hasn't dropped yet
            predictionsWithPlaceHolders.append(placeHolders1.pop())
        else: # candidate has dropped
            predictionsWithPlaceHolders.append(candidate)
                
    for i, candidate in enumerate(playerPredictionsArray):
        if i < len(dropOutOrder):
            dropsWithPlaceHolders.append(dropOutOrder[i])
        else:
            dropsWithPlaceHolders.append(placeHolders2.pop())
    
    return calculateKendallTauDistance(dropsWithPlaceHolders, predictionsWithPlaceHolders);

df = pd.read_csv("../2024/leaderboard/data/submissions2024AllColumns.csv")
df = df[['leaderboardAlias', 'prediction']]

## Calculate 'average' drop out position of each candidate
candidateScores = {}
for i in df.index:
    prediction = df['prediction'][i].split(",")[:-1]
    for i, candidate in enumerate(prediction):
        if candidate in candidateScores:
            candidateScores[candidate] += i
        else:
            candidateScores[candidate] = i

averagePredictions = [{'candidate': key, 'value': candidateScores[key] / len(df)} for key in candidateScores.keys()]
averagePredictions.sort(key=lambda x: x['value'])

averageOrder = [ap['candidate'] for ap in averagePredictions]

display(averagePredictions)
print("The 'Wisdom Of The Crowd' predicted drop out order is:")
display(averageOrder)

# Add average answer to submissions
predictionStr = ''
for p in averageOrder:
    predictionStr += p + ','
aggregateRow = {'leaderboardAlias':['Wisdom of the crowd'], 'prediction':[predictionStr]}
aggregateRow = pd.DataFrame(aggregateRow)
df = df.append(aggregateRow, ignore_index=True)

display(df.tail(1))

[{'candidate': 'hurd', 'value': 2.08974358974359},
 {'candidate': 'elder', 'value': 2.2564102564102564},
 {'candidate': 'suarez', 'value': 2.730769230769231},
 {'candidate': 'burgum', 'value': 3.3461538461538463},
 {'candidate': 'hutchinson', 'value': 3.3846153846153846},
 {'candidate': 'scott', 'value': 5.987179487179487},
 {'candidate': 'christie', 'value': 6.256410256410256},
 {'candidate': 'ramaswamy', 'value': 6.551282051282051},
 {'candidate': 'haley', 'value': 6.576923076923077},
 {'candidate': 'pence', 'value': 7.141025641025641},
 {'candidate': 'desantis', 'value': 9.076923076923077},
 {'candidate': 'trump', 'value': 10.602564102564102}]

The 'Wisdom Of The Crowd' predicted drop out order is:


['hurd',
 'elder',
 'suarez',
 'burgum',
 'hutchinson',
 'scott',
 'christie',
 'ramaswamy',
 'haley',
 'pence',
 'desantis',
 'trump']

,leaderboardAlias,prediction
78,Wisdom of the crowd,"hurd,elder,suarez,burgum,hutchinson,scott,chri..."


In [10]:
averageOrder = ['hurd','elder','suarez','burgum','hutchinson','scott','christie','ramaswamy','haley','pence','desantis','trump']
dropOutOrder = ['suarez', 'hurd', 'elder','pence','scott','burgum','christie','ramaswamy','hutchinson','desantis']

partialScoring(dropOutOrder, averageOrder)

13

In [11]:
# create a column for every previous and current round, so we can track score over time
for i in range(0, len(dropOutOrder)):
    df['round_' + str(i+1) + '_score'] = 0
    
# create a column for most recent round's score
df['kendallDistance'] = 0
    
# calc kendall distances for every previous round and the current round
for i in df.index:
    prediction = df['prediction'][i].split(",")[:-1]
    for round in range(0, len(dropOutOrder)):
        df['round_' + str(round+1) + '_score'][i] = partialScoring(dropOutOrder[:round+1], prediction)
    df['kendallDistance'][i] = partialScoring(dropOutOrder, prediction)
    
# for every distance column, create a rank column
df['rank'] = df['kendallDistance'].rank(method='min')

for i in range(0, len(dropOutOrder)):
    df['round_' + str(i+1) + '_rank'] = df['round_' + str(i+1) + '_score'].rank(method='min')

display(df.tail(1))

,leaderboardAlias,prediction,round_1_score,round_2_score,round_3_score,round_4_score,round_5_score,round_6_score,round_7_score,round_8_score,...,round_1_rank,round_2_rank,round_3_rank,round_4_rank,round_5_rank,round_6_rank,round_7_rank,round_8_rank,round_9_rank,round_10_rank
78,Wisdom of the crowd,"hurd,elder,suarez,burgum,hutchinson,scott,chri...",2,2,2,8,10,10,11,12,...,34.0,15.0,10.0,28.0,20.0,12.0,9.0,6.0,6.0,6.0


In [14]:
df.to_csv("../2024/leaderboard/data/submissions2024.csv", index=False)

In [18]:
33 / (len(df) - 1) * 100

42.30769230769231

In [19]:
8 / (len(df) - 1) * 100

10.256410256410255